In [1]:
import pandas as pd
import psycopg2 
# Conectar ao banco de dados PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="dw_covid",
    user="postgres",
    password="1234"
)
cursor = conn.cursor()

In [2]:
df = pd.read_csv("D:\\covid-data-analysis\\covid-modified.csv", delimiter =";")
df.columns = df.columns.str.lower()

In [3]:
conn.rollback()

In [8]:
for index, row in df.iterrows():
    try:
        cursor.execute("SELECT id_geography FROM dim_geography WHERE country = %s AND country_code = %s AND region = %s", (row['country'], row['country_code'], row['region']))
        geography_id = cursor.fetchone()
        geography_id = geography_id[0] if geography_id else None

        cursor.execute("SELECT id_date FROM dim_date WHERE date_reported = %s", (row['date_reported'],))
        date_id = cursor.fetchone()
        date_id = date_id[0] if date_id else None

        cursor.execute("""
            INSERT INTO fact_covid (
                new_cases, cumulative_cases, new_deaths, cumulative_deaths, id_geography, id_date
            ) VALUES (%s, %s, %s, %s, %s, %s)
            """, (
                row['new_cases'], row['cumulative_cases'], row['new_deaths'], row['cumulative_deaths'], geography_id, date_id
            ))

        conn.commit()

    except Exception as e:
        print(f"Erro encontrado: {e}")
        conn.rollback()
